In [1]:
import sys
import os
sys.path.append('..')
# sys.path.append(os.path.abspath('..'))

from incf.incf import MainArea
import param

import io
import panel as pn
import numpy as np
import pandas as pd

pn.extension('tabulator', 'ace', 'jsoneditor', 'ipywidgets', sizing_mode='stretch_width', notifications=True)

NotificationArea(sizing_mode='stretch_width')

In [2]:
from pathlib import Path

import warnings
warnings.filterwarnings('ignore')

In [3]:
path = '../output'
sys.path.append(path)

In [4]:
test = MainArea()
pn.serve(test.view())


Launching server at http://localhost:55069


In [5]:
def get_current_output(path, indentation=2):
    tree = []

    if os.path.exists(path) and len(os.listdir(path)) > 0:
        for root, dirs, files in os.walk(path):
            level = root.replace(path, '').count(os.sep)
            indent = '&emsp;' * indentation * level
            tree.append('{}{}/'.format(indent, os.path.basename(root)))
        tree = '<br>'.join(tree)
    return tree


get_current_output('../output')

'output/<br>&emsp;&emsp;code/<br>&emsp;&emsp;coord/<br>&emsp;&emsp;eq/<br>&emsp;&emsp;param/<br>&emsp;&emsp;sub-10qA0/<br>&emsp;&emsp;&emsp;&emsp;net/<br>&emsp;&emsp;&emsp;&emsp;spatial/<br>&emsp;&emsp;&emsp;&emsp;ts/<br>&emsp;&emsp;sub-kh0iA/<br>&emsp;&emsp;&emsp;&emsp;net/<br>&emsp;&emsp;&emsp;&emsp;spatial/<br>&emsp;&emsp;&emsp;&emsp;ts/<br>&emsp;&emsp;sub-RuBcx/<br>&emsp;&emsp;&emsp;&emsp;net/<br>&emsp;&emsp;&emsp;&emsp;spatial/<br>&emsp;&emsp;&emsp;&emsp;ts/'

In [12]:
from pathlib import Path
from itertools import islice

space =  '    '
branch = '│   '
tee =    '├── '
last =   '└── '



def tree(dir_path: Path, level: int=-1, limit_to_directories: bool=False,
         length_limit: int=1000):
    """Given a directory Path object print a visual tree structure"""
    struct = []
    
    dir_path = Path(dir_path) # accept string coerceable to Path
    files = 0
    directories = 0
    def inner(dir_path: Path, prefix: str='', level=-1):
        nonlocal files, directories
        if not level: 
            return # 0, stop iterating
        if limit_to_directories:
            contents = [d for d in dir_path.iterdir() if d.is_dir()]
        else: 
            contents = list(dir_path.iterdir())
        pointers = [tee] * (len(contents) - 1) + [last]
        for pointer, path in zip(pointers, contents):
            if path.is_dir():
                yield prefix + pointer + path.name
                directories += 1
                extension = branch if pointer == tee else space 
                yield from inner(path, prefix=prefix+extension, level=level-1)
            elif not limit_to_directories:
                yield prefix + pointer + path.name
                files += 1
    struct.append(dir_path.name)
    
    
    iterator = inner(dir_path, level=level)
    for line in islice(iterator, length_limit):
        struct.append(line)
    return '<br>'.join(struct)
    

tree('../output')

'output<br>├── CHANGES.txt<br>├── code<br>├── coord<br>│   ├── desc-default_connectivity_labels.json<br>│   ├── desc-default_connectivity_labels.tsv<br>│   ├── desc-default_connectivity_nodes.json<br>│   └── desc-default_connectivity_nodes.tsv<br>├── dataset_description.json<br>├── eq<br>├── param<br>│   └── desc-default-generic2doscillator.xml<br>├── participants.tsv<br>├── README.txt<br>├── sub-10qA0<br>│   ├── net<br>│   ├── spatial<br>│   └── ts<br>├── sub-kh0iA<br>│   ├── net<br>│   │   ├── sub-kh0iA_desc-default_connectivity-distances.json<br>│   │   ├── sub-kh0iA_desc-default_connectivity-distances.tsv<br>│   │   ├── sub-kh0iA_desc-default_connectivity-weights.json<br>│   │   └── sub-kh0iA_desc-default_connectivity-weights.tsv<br>│   ├── spatial<br>│   └── ts<br>└── sub-RuBcx<br>    ├── net<br>    │   ├── sub-RuBcx_desc-default_connectivity-distances.json<br>    │   ├── sub-RuBcx_desc-default_connectivity-distances.tsv<br>    │   ├── sub-RuBcx_desc-default_connectivity-weights.j

In [46]:
import pprint

pp = pprint.PrettyPrinter()

def path_to_dict(path, d={'dirs':{},'files':[]}):
    
    name = os.path.basename(path)

    if os.path.isdir(path):
        if name not in d['dirs']:
            d['dirs'][name] = {'dirs':{},'files':[]}
        for x in os.listdir(path):
            path_to_dict(os.path.join(path,x), d['dirs'][name])
    else:
        d['files'].append(name)
        
    return d['dirs']


mydict = path_to_dict('../output')
mydict

{'output': {'dirs': {'code': {'dirs': {}, 'files': []},
   'coord': {'dirs': {},
    'files': ['desc-default_connectivity_labels.json',
     'desc-default_connectivity_labels.tsv',
     'desc-default_connectivity_nodes.json',
     'desc-default_connectivity_nodes.tsv']},
   'eq': {'dirs': {}, 'files': []},
   'param': {'dirs': {}, 'files': ['desc-default-generic2doscillator.xml']},
   'sub-10qA0': {'dirs': {'net': {'dirs': {}, 'files': []},
     'spatial': {'dirs': {}, 'files': []},
     'ts': {'dirs': {}, 'files': []}},
    'files': []},
   'sub-kh0iA': {'dirs': {'net': {'dirs': {},
      'files': ['sub-kh0iA_desc-default_connectivity-distances.json',
       'sub-kh0iA_desc-default_connectivity-distances.tsv',
       'sub-kh0iA_desc-default_connectivity-weights.json',
       'sub-kh0iA_desc-default_connectivity-weights.tsv']},
     'spatial': {'dirs': {}, 'files': []},
     'ts': {'dirs': {}, 'files': []}},
    'files': []},
   'sub-RuBcx': {'dirs': {'net': {'dirs': {},
      'files':

In [45]:
output = list(mydict.keys())[0]
mydict[output]
# output

{'dirs': {'code': {'dirs': {}, 'files': []},
  'coord': {'dirs': {},
   'files': ['desc-default_connectivity_labels.json',
    'desc-default_connectivity_labels.tsv',
    'desc-default_connectivity_nodes.json',
    'desc-default_connectivity_nodes.tsv']},
  'eq': {'dirs': {}, 'files': []},
  'param': {'dirs': {}, 'files': ['desc-default-generic2doscillator.xml']},
  'sub-10qA0': {'dirs': {'net': {'dirs': {}, 'files': []},
    'spatial': {'dirs': {}, 'files': []},
    'ts': {'dirs': {}, 'files': []}},
   'files': []},
  'sub-kh0iA': {'dirs': {'net': {'dirs': {},
     'files': ['sub-kh0iA_desc-default_connectivity-distances.json',
      'sub-kh0iA_desc-default_connectivity-distances.tsv',
      'sub-kh0iA_desc-default_connectivity-weights.json',
      'sub-kh0iA_desc-default_connectivity-weights.tsv']},
    'spatial': {'dirs': {}, 'files': []},
    'ts': {'dirs': {}, 'files': []}},
   'files': []},
  'sub-RuBcx': {'dirs': {'net': {'dirs': {},
     'files': ['sub-RuBcx_desc-default_connec

In [ ]:
def get_structure(path, files):
    struct = path_to_dict('../output')
    output = list(struct.keys())[0]
    struct = struct[output]
    
    add_structure(struct, files)
    

def add_structure(struct, files):
    for file in files:
        
    
    

In [ ]:
def create_layout(subs=None, output='../output'):
    """
    Create folder structure according to BEP034 and passed `subs` parameter.

    :param output:
    :param subs:
    :return:
    """

    output = output.replace('.', '').replace('/', '')
    layout = create_sub(subs)
    layout = '&emsp;'.join(layout) if len(layout) > 1 else ''.join(layout)

    return f"""
    {output}/ <br>
        &emsp;|___ code <br>
        &emsp;|___ eq <br>
        &emsp;|___ param <br>
        &emsp;{layout}
        &emsp;|___ README <br>
        &emsp;|___ CHANGES <br>
        &emsp;|___ dataset_description.json <br>
        &emsp;|___ participants.tsv
    """

In [25]:
def showFolderTree(path, indentation=2):
    """
    Shows the content of a folder in a tree structure.
    path -(string)- path of the root folder we want to show.
    show_files -(boolean)-  Whether or not we want to see files listed.
                            Defaults to False.
    indentation -(int)- Indentation we want to use, defaults to 2.   
    file_output -(string)-  Path (including the name) of the file where we want
                            to save the tree.
    """


    tree = []

    for root, dirs, files in os.walk(path):
        level = root.replace(path, '').count(os.sep)
        indent = '&emsp;'*indentation*(level)
        tree.append('{}{}/'.format(indent, os.path.basename(root)))
    return tree

            
showFolderTree('../output')

['output/',
 '&emsp;&emsp;code/',
 '&emsp;&emsp;coord/',
 '&emsp;&emsp;eq/',
 '&emsp;&emsp;param/',
 '&emsp;&emsp;sub-10qA0/',
 '&emsp;&emsp;&emsp;&emsp;net/',
 '&emsp;&emsp;&emsp;&emsp;spatial/',
 '&emsp;&emsp;&emsp;&emsp;ts/',
 '&emsp;&emsp;sub-kh0iA/',
 '&emsp;&emsp;&emsp;&emsp;net/',
 '&emsp;&emsp;&emsp;&emsp;spatial/',
 '&emsp;&emsp;&emsp;&emsp;ts/',
 '&emsp;&emsp;sub-RuBcx/',
 '&emsp;&emsp;&emsp;&emsp;net/',
 '&emsp;&emsp;&emsp;&emsp;spatial/',
 '&emsp;&emsp;&emsp;&emsp;ts/']

In [30]:
def get_folder_contents(path, subs):
    subs[path] = os.listdir()
    
    for file in os.listdir(path):
        if os.path.isdir(os.path.join(path, file)):
             get_folder_contents(os.path.join(path, file), subs)
            
        


def get_contents(path):
    subs = OrderedDict({'main': []})
    
    for file in os.listdir(path):
        dir_path = os.path.join(path, file)
        if os.path.isdir(dir_path):
            print(get_folder_contents(dir_path, subs))
            
        else:
            subs['main'].append(file)
            
    print(subs)
        
        
get_contents('../output')

None
None
None
None
None
None
None
OrderedDict([('main', ['CHANGES.txt', 'dataset_description.json', 'participants.tsv', 'README.txt']), ('../output\\coord', 'desc-default_connectivity_nodes.tsv'), ('../output\\param', 'desc-default-generic2doscillator.xml')])


In [14]:
from collections import OrderedDict

def get_current_files(path):
    file_count, dir_count = 0, 0
    struct = OrderedDict()
    
    for root, dirs, files in os.walk(path):
        level = root.replace(path, '').count(os.sep)
        indent = ' ' * 4 * level
        subindent = ' ' * 4 * (level + 1)
        
        name = os.path.basename(root)
        struct[name] = []
        struct[name].append('{}{}\n'.format(indent, name))
        
        for file in files:
            struct[name].append('{}{}'.format(subindent, file))
            file_count += 1
        dir_count += len(dirs)
        
    return struct
        
get_current_files('../output')

OrderedDict([('output',
              ['output\n',
               '    CHANGES.txt',
               '    dataset_description.json',
               '    participants.tsv',
               '    README.txt']),
             ('code', ['    code\n']),
             ('coord',
              ['    coord\n',
               '        desc-default_connectivity_labels.json',
               '        desc-default_connectivity_labels.tsv',
               '        desc-default_connectivity_nodes.json',
               '        desc-default_connectivity_nodes.tsv']),
             ('eq', ['    eq\n']),
             ('param',
              ['    param\n', '        desc-default-generic2doscillator.xml']),
             ('sub-10qA0', ['    sub-10qA0\n']),
             ('net',
              ['        net\n',
               '            sub-RuBcx_desc-default_connectivity-distances.json',
               '            sub-RuBcx_desc-default_connectivity-distances.tsv',
               '            sub-RuBcx_desc-defa

In [58]:
def list_files(startpath, n_indent=2):
    file_count, dir_count = 0, 0
    struct = []
    
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ('&emsp;' * n_indent * level) + '|___'
        subindent = ('&emsp;' * n_indent * (level + 1)) + '|___'
        
        # append values
        struct.append('{}{}/'.format(indent, os.path.basename(root)))
        
        # iterate over content and append values
        for f in files:
            struct.append('{}{}'.format(subindent, f))
            file_count += 1
        dir_count += len(dirs)
        
    txt = f'<br>{dir_count} directories, {file_count} files'
    return struct, txt


In [6]:
def append_files(path, include=None):
    """
    Append all files in a specified file using recursive walk. It implements a
    top-down approach of traversing the directory.

    :param path:
    :param include:
    :return:
    """

    all_files = []
    for root, dirs, files in os.walk(path, topdown=True):
        for file in files:
            if include is None:
                all_files.append(os.path.basename(file))
            else:
                if file in include:
                    all_files.append(os.path.join(root, file))
    return all_files


append_files('C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\1')

['weights.txt']

In [12]:
append_files('C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files\\2')

['centres.txt', 'tract_lengths.txt', 'weights.txt']

In [14]:
import functools

def get_directory_structure(rootdir):
    """
    Creates a nested dictionary that represents the folder structure of rootdir
    """
    dir = {}
    rootdir = rootdir.rstrip(os.sep)
    start = rootdir.rfind(os.sep) + 1
    for path, dirs, files in os.walk(rootdir):
        folders = path[start:].split(os.sep)
        print(f'folders: {folders}')
        subdir = dict.fromkeys(files)
        print(f'subdir: {subdir}')
        parent = functools.reduce(dict.get, folders[:-1], dir)
        print(f'parent: {parent}')
        parent[folders[-1]] = subdir
    return dir

In [15]:
get_directory_structure('C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files')

folders: ['txt_files']
subdir: {}
parent: {}
folders: ['txt_files', '1']
subdir: {'weights.txt': None}
parent: {}
folders: ['txt_files', '2']
subdir: {'centres.txt': None, 'tract_lengths.txt': None, 'weights.txt': None}
parent: {'1': {'weights.txt': None}}


{'txt_files': {'1': {'weights.txt': None},
  '2': {'centres.txt': None, 'tract_lengths.txt': None, 'weights.txt': None}}}

In [20]:
import functools

def get_directory_structure(rootdir):
    """
    Creates a nested dictionary that represents the folder structure of rootdir
    """
    dir = {}
    rootdir = rootdir.rstrip(os.sep)
    start = rootdir.rfind(os.sep) + 1
    
    for path, dirs, files in os.walk(rootdir):
        folders = path[start:].split(os.sep)
        subdir = {}
        
        for file in files:
            print(os.path.join(rootdir, file))
            if os.path.isdir(os.path.join(rootdir, file)):
                print(True, file)
        
        
        
        print(f'subdir: {subdir}')
#         parent = functools.reduce(dict.get, folders[:-1], dir)
#         print(f'parent: {parent}')
#         parent[folders[-1]] = subdir
    return dir

In [21]:
get_directory_structure('C:\\Users\\dinar\\Desktop\\gsoc\\data\\txt_files')

subdir: {}
C:\Users\dinar\Desktop\gsoc\data\txt_files\weights.txt
subdir: {}
C:\Users\dinar\Desktop\gsoc\data\txt_files\centres.txt
C:\Users\dinar\Desktop\gsoc\data\txt_files\tract_lengths.txt
C:\Users\dinar\Desktop\gsoc\data\txt_files\weights.txt
subdir: {}


{}

In [13]:
# initializing list
lis = [1, 3, 5, 6, 2, ]
 
# using reduce to compute sum of list
print("The sum of the list elements is : ", end="")
print(functools.reduce(lambda a, b: a+b, lis))

The sum of the list elements is : 17
<function <lambda> at 0x00000255B5C692D0> [1, 3, 5, 6, 2]


In [ ]:
def dir_walk(path):
    all_files = {}
    r, d = None, None
    cur_dir = None

    for root, dirs, files in os.walk(path, topdown=True):
        if r is None:
            r = root


#         rname = os.path.basename(root)

#         all_files[rname] = {'files': [], 'dirs': []}

#         for d in dirs:
#             all_files[rname]['dirs'].append(os.path.join(root, d))
#         for f in files:
#             all_files[rname]['files'].append(os.path.join(root, f))
#         for file in files:
#             print()

#         for file in files:
#             print(os.path.join(root, file))
    return all_files



In [1]:
def dir_walk(path):
    all_files = {}
    r, d = None, None
    cur_dir = None
    
    for root, dirs, files in os.walk(path, topdown=True):
        if r is None:
            r = root
            
        
#         rname = os.path.basename(root)
        
#         all_files[rname] = {'files': [], 'dirs': []}
        
#         for d in dirs:
#             all_files[rname]['dirs'].append(os.path.join(root, d))
#         for f in files:
#             all_files[rname]['files'].append(os.path.join(root, f))
#         for file in files:
#             print()
            
#         for file in files:
#             print(os.path.join(root, file))
    return all_files
            
            

In [2]:
dir_walk('C:\\Users\\dinar\\Desktop\\gsoc\\data')

NameError: name 'os' is not defined

In [64]:
dir_walk('C:\\Users\\dinar\\Desktop\\gsoc\\data\\simulations\\1')

C:\Users\dinar\Desktop\gsoc\data\simulations\1\datatypes\Connectivity_6b28b6c61836452bb52cbd18538a7775.h5
C:\Users\dinar\Desktop\gsoc\data\simulations\1\BurstConfiguration_a29116b38d05429588535667ce4d1eee.h5
C:\Users\dinar\Desktop\gsoc\data\simulations\1\Generic2dOscillator_4e4d2edc01e2448192baf025336e0d73.h5
C:\Users\dinar\Desktop\gsoc\data\simulations\1\HeunDeterministicViewModel_4007260a59a443cfbc076eb4c231c6f5.h5
C:\Users\dinar\Desktop\gsoc\data\simulations\1\Linear_f639e3b95c48436dae5ffaa29072963e.h5
C:\Users\dinar\Desktop\gsoc\data\simulations\1\SimulatorAdapterModel_994a4afbef39461abf2a4feaf9761ca0.h5
C:\Users\dinar\Desktop\gsoc\data\simulations\1\SubSampleViewModel_cd2fe7f44797434b9ae2cc3418a9f091.h5


{}

In [65]:
class Settings(param.Parameterized):
    sub_options = ['Single subject', 'Multiple subjects']
    checkbox_options = ['Traverse sub-folders', ]

    sub_select = pn.widgets.RadioButtonGroup(options=sub_options, button_type='default',
                                             value=[], margin=(-20, 0, 0, 0))
    checkbox = pn.widgets.CheckBoxGroup(value=[], options=checkbox_options)

    @pn.depends('sub_select.value', watch=True)
    def _change_selection(self):
        if self.sub_select.value == self.sub_options[0]:
            # TODO: add all the files to one folder
            print('Selected single value')
#             pass
        else:
            # TODO: infer which relates to which
            pass
        pass

    def view(self):
        return pn.Column(
            '## Settings',
            '#### Select subject count',
            self.sub_select
        )


pn.serve(Settings().view())

C:\Users\dinar\Desktop\gsoc\data\timeseries\1\timeseries_all.mat
C:\Users\dinar\Desktop\gsoc\data\timeseries\2\timeseries_all.mat
C:\Users\dinar\Desktop\gsoc\data\timeseries\3\timeseries_all.mat
C:\Users\dinar\Desktop\gsoc\data\timeseries\4\timeseries_all.mat
C:\Users\dinar\Desktop\gsoc\data\timeseries\5\timeseries_all.mat


{}

In [8]:
class Settings(param.Parameterized):
    sub_options = ['Single subject', 'Multiple subjects']
    checkbox_options = ['Traverse sub-folders', ]
    
    sub_select = pn.widgets.RadioButtonGroup(options=sub_options, button_type='default',
                                             value=[], margin=(-20, 0, 0, 0))
    checkbox = pn.widgets.CheckBoxGroup(value=[], options=checkbox_options)

    @pn.depends('sub_select.value', watch=True)
    def _change_selection(self):
        if self.sub_select.value == self.sub_options[0]:
            # TODO: add all the files to one folder
            print('Selected single value')
#             pass
        else:
            # TODO: infer which relates to which
            pass 
        pass

    def view(self):
        return pn.Column(
            '## Settings',
            '#### Select subject count',
            self.sub_select
        )

    
pn.serve(Settings().view())

Launching server at http://localhost:58960


Selected single value


In [ ]:
checkbox_group = pn.widgets.CheckBoxGroup(
    name='Checkbox Group', value=['Apple', 'Pear'], options=['Apple', 'Banana', 'Pear', 'Strawberry'],
    inline=True)

In [ ]:
def get_files(path='../output', ftype='.json'):
    f = []

    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            if ftype in name:
                f.append(os.path.join(root, name))
    return f


class ViewResults(param.Parameterized):
    options = ['JSON files', 'TSV files']
    file_selection = pn.widgets.RadioButtonGroup(options=options, button_type='primary', value=[])
#     widget = pn.WidgetBox('### Select File', select_options)

    def __init__(self, path):
        super().__init__()
        self.path = path
        self.layout = None
        self.select_options = pn.widgets.Select()
        self.widget = pn.WidgetBox('### Select File', self.select_options)

    @pn.depends('file_selection.value', watch=True)
    def _change_filetype(self):
        print(self.file_selection.value)
        if self.file_selection.value == 'JSON files':
            self.select_options.options = get_files(path=self.path)
            self.change_file()
        elif self.file_selection.value == 'TSV files':
            self.select_options.options = get_files(path=self.path, ftype='.tsv')
            self.change_file()

#     @pn.depends('select_options.value', watch=True)
    def change_file(self):
        if len(self.widget) > 2:
            self.widget.pop(-1)

        if self.file_selection.value == 'JSON files':
            try:
                file = json.load(open(self.select_options.value))
            except Exception:
                print(f'File `{self.select_options.value}` is empty!')
            else:
                self.widget.append(pn.widgets.JSONEditor(value=file, height=350))

        elif self.file_selection.value == 'TSV files':
            try:
                file = pd.read_csv(self.select_options.value, sep='\t', header=None, index_col=None)
            except Exception:
                print(f'File `{self.select_options.value}` is empty!')
            else:
                self.widget.append(pn.widgets.Tabulator(file))

    def view(self):
        return pn.Column(self.file_selection, self.widget)


pn.serve(ViewResults('../output').view())

In [4]:
pn.state.notifications.error('This is an error notification.', duration=1000)
pn.state.notifications.info('This is a info notification.', duration=2000)
pn.state.notifications.success('This is a success notification.')
pn.state.notifications.warning('This is a warning notification.', duration=4000);

In [15]:
def get_files(path='../output', ftype='.json'):
    f = []

    for root, dirs, files in os.walk(path, topdown=False):
        for name in files:
            if ftype in name:
                f.append(os.path.join(root, name))
    return f


class ViewResults(param.Parameterized):
    options = ['JSON files', 'TSV files']
    file_selection = pn.widgets.RadioButtonGroup(options=options, button_type='primary', value=[])
#     widget = pn.WidgetBox('### Select File', select_options)
    
    def __init__(self, path):
        super().__init__()
        self.path = path
        self.layout = None 
        self.select_options = pn.widgets.Select()
        self.widget = pn.WidgetBox('### Select File', self.select_options)

    @pn.depends('file_selection.value', watch=True)
    def _change_filetype(self):
        print(self.file_selection.value)
        if self.file_selection.value == 'JSON files':
            self.select_options.options = get_files(path=self.path)
            self.change_file()
        elif self.file_selection.value == 'TSV files':
            self.select_options.options = get_files(path=self.path, ftype='.tsv')
            self.change_file()

#     @pn.depends('select_options.value', watch=True)
    def change_file(self):
        if len(self.widget) > 2:
            self.widget.pop(-1)

        if self.file_selection.value == 'JSON files':
            try:
                file = json.load(open(self.select_options.value))
            except Exception:
                print(f'File `{self.select_options.value}` is empty!')
            else:
                self.widget.append(pn.widgets.JSONEditor(value=file, height=350))

        elif self.file_selection.value == 'TSV files':
            try:
                file = pd.read_csv(self.select_options.value, sep='\t', header=None, index_col=None)
            except Exception:
                print(f'File `{self.select_options.value}` is empty!')
            else:
                self.widget.append(pn.widgets.Tabulator(file))

    def view(self):
        return pn.Column(self.file_selection, self.widget)


pn.serve(ViewResults('../output').view())

Launching server at http://localhost:53762


None
JSON files
File `../output\coord\desc-default_labels.json` is empty!


In [7]:
cs = pn.widgets.CrossSelector(width=700)
pn.serve(cs)

Launching server at http://localhost:50123


In [12]:
string = param.String('hehe')
dir(string)

['__class__',
 '__classdoc',
 '__delattr__',
 '__delete__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__set__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_internal_name',
 '_label',
 '_on_set',
 '_post_setter',
 '_serializers',
 '_set_instantiate',
 '_set_names',
 '_validate',
 '_validate_regex',
 '_validate_value',
 'allow_None',
 'constant',
 'default',
 'deserialize',
 'doc',
 'instantiate',
 'label',
 'name',
 'owner',
 'per_instance',
 'pickle_default_value',
 'precedence',
 'readonly',
 'regex',
 'schema',
 'serialize',
 'watchers']

In [18]:
string.default

'hehe'

In [19]:
string = 'm'

In [21]:
string

AttributeError: 'str' object has no attribute 'default'

### Check if there's a difference between empirical FC & simulated data

In [4]:
from scipy.io import loadmat
import mat73

In [8]:
sim = loadmat("C:\\Users\\dinar\\Desktop\\simulated_data\\1\\AY_20111004_SC_new.mat\\cspeed20\\csf 0.05\\bold.mat", squeeze_me=True)
sim

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Tue Feb  3 22:52:09 2015',
 '__version__': '1.0',
 '__globals__': [],
 'arr': array([[-110.52488034, -445.28196601, -130.27069805, ...,  488.25445135,
          343.85266138,  400.69137436],
        [-110.88021998, -445.46276722, -130.90696322, ...,  486.99161812,
          342.56528117,  399.46318728],
        [-113.08478947, -446.57163538, -134.26416291, ...,  478.1312449 ,
          334.87214655,  391.93338033],
        ...,
        [-621.6660361 , -627.18351817, -763.04972688, ..., -526.41360245,
         -742.10168801, -753.38319788],
        [-611.04317113, -614.24401962, -776.81508901, ..., -541.60214243,
         -747.9391402 , -755.31800616],
        [-600.79134046, -599.38352235, -785.59528833, ..., -552.76523283,
         -742.87992851, -759.7311778 ]])}

In [16]:
np.percentile(sim['arr'], [0, 50, 100])

array([-1177.56475519,  -617.04150771,   588.59473112])

File `None` is empty!
Creating folder `output`...
Creating folder `eq`...
Creating folder `code`...
Creating folder `coord`...
Creating folder `param`...
Creating file `README.txt`...
Creating file `participants.tsv`...
Creating file `dataset_description.json`...
Creating file `CHANGES.txt`...
Creating folder `../output\sub-dD8vA`
Creating folder `../output\sub-dD8vA\net`
Creating folder `../output\sub-dD8vA\spatial`
Creating folder `../output\sub-dD8vA\ts`


In [8]:
data = loadmat('C:\\Users\\dinar\\Desktop\\gsoc\\data\\timeseries\\1\\timeseries_all.mat')

np.percentile(data['data'], [0, 50, 100])

array([ 5657.60373 , 11453.467125, 18926.23528 ])

In [15]:
emp_fc = loadmat("C:\\Users\\dinar\\Desktop\\simulated_data\\1\\all_empFC.mat")
np.percentile(emp_fc['all_empFC'], [0, 50, 100])

array([-0.64908182,  0.44813303,  1.        ])

In [11]:
loadmat('C:\\Users\\dinar\\Desktop\\gsoc\\data\\ses-preop\\FC.mat')

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 4.0.2, 2017-02-21 10:35:45 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'CON01T1_ROIts': array([[  923.70188, 10967.75413, 10075.33814, ...,  9686.10222,
         10282.49023, 10792.33884],
        [  916.27902, 10939.30918,  9958.87689, ...,  9665.7039 ,
         10121.32577, 10689.26117],
        [  906.29391, 10909.99712,  9918.38669, ...,  9593.44135,
         10135.87439, 10713.51698],
        ...,
        [  911.26295, 10948.92151,  9881.90784, ...,  9584.80664,
         10276.75591, 10826.52015],
        [  911.89234, 10926.43679,  9956.72355, ...,  9550.86363,
         10231.36131, 10768.50695],
        [  914.11457, 10917.22143,  9884.80337, ...,  9557.05501,
         10139.23922, 10715.65225]]),
 'CON01T1_ROIts_DK68': array([[12974.08918, 11915.09521, 12078.56191, ...,  9686.10222,
         10282.49023, 10792.33884],
        [12959.45346, 11819.84904, 12055.11605, ...,  9665.7039 ,
         10121.32577, 10689.26117]

### speed20, etc

In [19]:
data = loadmat('C:\\Users\\dinar\\Desktop\\simulated_data\\2\\first_exploration\\AA_20120815_log_SC_csf_0.0001_speed_50.mat',
               squeeze_me=True)
data

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Fri Sep 29 11:46:43 2017',
 '__version__': '1.0',
 '__globals__': [],
 'subs_data': array([[[ 0.51877537,  0.32200462,  0.62658588, ...,  0.60863629,
           0.568951  ,  0.78985981],
         [ 0.58101576,  0.61059093,  0.40993171, ...,  0.55985378,
           0.43702166,  0.40615905]],
 
        [[-1.53511119, -0.49331051, -0.53308641, ..., -0.51905395,
          -0.55381113,  0.04665336],
         [-0.50735542, -0.4094834 , -0.41427879, ...,  0.08833556,
          -0.46056716, -0.46410892]],
 
        [[-1.04546626, -0.35251508, -0.41500527, ..., -0.20582285,
          -1.25996799, -0.7764147 ],
         [-0.34540655, -0.32373849, -0.34252525, ..., -0.39158608,
          -0.34193913, -0.36278282]],
 
        ...,
 
        [[ 0.77479767, -0.98001667, -0.84482694, ..., -1.28450549,
          -1.14173223,  1.56361209],
         [ 1.223174  , -0.36430809, -0.34337304, ...,  0.10573174,
          -0.64746735,  0.495819

In [20]:
data['subs_time'].shape, data['subs_data'].shape

((1500,), (1500, 2, 68))

In [23]:
data['subs_data'][0]

array([[0.51877537, 0.32200462, 0.62658588, 0.41850138, 0.26502229,
        0.43742028, 0.48043986, 0.15356728, 0.54771136, 0.59302886,
        0.48912306, 0.74123135, 0.48709837, 0.37643532, 0.58197464,
        0.96119918, 0.72236114, 0.30243342, 0.21796086, 0.42015013,
        0.31026511, 0.56846824, 0.58036834, 0.56682696, 0.86364423,
        0.02419162, 0.70922303, 0.33941018, 0.40719929, 0.24735968,
        0.65620849, 0.35796918, 0.58750035, 0.47030968, 0.44316665,
        0.53620101, 0.6096645 , 0.14700385, 0.51637336, 0.6265656 ,
        0.50779293, 0.64715845, 0.53941423, 0.3632348 , 0.41904449,
        0.03918741, 0.32387129, 0.75251248, 0.42285177, 0.37035287,
        0.6414281 , 0.56651524, 0.53169924, 0.58234798, 0.73980684,
        0.47107773, 0.77864327, 0.54553439, 0.62377154, 0.1393394 ,
        0.33361236, 0.66322912, 0.436849  , 0.3422435 , 0.48019297,
        0.60863629, 0.568951  , 0.78985981],
       [0.58101576, 0.61059093, 0.40993171, 0.48334802, 0.71842177,
   

In [65]:
os.listdir('C:\\Users\\dinar\\Desktop\\gsoc\\data\\simulations\\1')

['BurstConfiguration_a29116b38d05429588535667ce4d1eee.h5',
 'datatypes',
 'Generic2dOscillator_4e4d2edc01e2448192baf025336e0d73.h5',
 'HeunDeterministicViewModel_4007260a59a443cfbc076eb4c231c6f5.h5',
 'Linear_f639e3b95c48436dae5ffaa29072963e.h5',
 'SimulatorAdapterModel_994a4afbef39461abf2a4feaf9761ca0.h5',
 'SubSampleViewModel_cd2fe7f44797434b9ae2cc3418a9f091.h5']

In [13]:
path = 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\simulations\\1'

for h5 in os.listdir(path):
    if h5 == 'datatypes': continue

    print('\n', h5, '\n\t', h5py.File(os.path.join(path, h5)).keys())


 BurstConfiguration_a29116b38d05429588535667ce4d1eee.h5 
	 <KeysViewHDF5 []>

 Generic2dOscillator_4e4d2edc01e2448192baf025336e0d73.h5 
	 <KeysViewHDF5 ['I', 'a', 'alpha', 'b', 'beta', 'c', 'd', 'e', 'f', 'g', 'gamma', 'tau']>

 HeunDeterministicViewModel_4007260a59a443cfbc076eb4c231c6f5.h5 
	 <KeysViewHDF5 []>

 Linear_f639e3b95c48436dae5ffaa29072963e.h5 
	 <KeysViewHDF5 ['a', 'b']>

 SimulatorAdapterModel_994a4afbef39461abf2a4feaf9761ca0.h5 
	 <KeysViewHDF5 []>

 SubSampleViewModel_cd2fe7f44797434b9ae2cc3418a9f091.h5 
	 <KeysViewHDF5 ['variables_of_interest']>


### Trial to save XML params

```
Generic2dOscillator_4e4d2edc01e2448192baf025336e0d73.h5
	 <KeysViewHDF5 ['I', 'a', 'alpha', 'b', 'beta', 'c', 'd', 'e', 'f', 'g', 'gamma', 'tau']>
```

In [16]:
import h5py
TXT_COLS = ['areas', 'centres', 'cortical', 'hemispheres', 'orientations', 'region_labels', 'tract_lengths', 'weights']


class XML:
    def __init__(self, path, output):
        self.path = path
        self.params = []
        self.output = output
        self.template = """<?xml version="1.0" ?>
<Lems xmlns="http://www.neuroml.org/lems/0.7.6" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.neuroml.org/lems/0.7.6 https://raw.githubusercontent.com/LEMS/LEMS/development/Schemas/LEMS/LEMS_v0.7.6.xsd">
    <Component {}/>
</Lems>"""
        self.parse_file()

    def parse_file(self):
        if self.path.endswith('.h5'):
            self.file = h5py.File(self.path)
            self.keys = list(self.file.keys())
            self.eq = list(self.check_keys())

            for k in self.eq:
                self.params.append('{}="{}"'.format(k, self.file[k][:][0]))

            if len(self.params) > 0:
                print(f'Found following parameters:\n{self.params}')
                self.populate_template()

    def check_keys(self):
        return set(self.keys).difference(set(TXT_COLS))

    def populate_template(self):
        if len(self.params) > 0:
            self.template = self.template.format(' '.join(self.params))
            self.create_xml()

    def create_xml(self):
        with open(self.output, 'w') as f:
            f.write(self.template)


xml = XML('C:\\Users\\dinar\\Desktop\\gsoc\\data\\simulations\\1\\Generic2dOscillator_4e4d2edc01e2448192baf025336e0d73.h5',
          'generic2doscillator.xml')

Found following parameters:
['d="0.02"', 'alpha="1.0"', 'b="-10.0"', 'e="3.0"', 'g="0.0"', 'gamma="1.0"', 'I="0.0"', 'f="1.0"', 'tau="1.0"', 'c="0.0"', 'beta="1.0"', 'a="-2.0"']


In [53]:
path = 'C:\\Users\\dinar\\Desktop\\gsoc\\data\\simulations\\1'

for idx, h5 in enumerate(os.listdir(path)):
    if h5 == 'datatypes': continue

    xml = XML(os.path.join(path, h5), f'test-{idx}')
    xml.parse_file()
    print(xml.template, end='\n\n\n')

Found following parameters:
[]
<?xml version="1.0" ?>
<Lems xmlns="http://www.neuroml.org/lems/0.7.6" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.neuroml.org/lems/0.7.6 https://raw.githubusercontent.com/LEMS/LEMS/development/Schemas/LEMS/LEMS_v0.7.6.xsd">
    <Component {}/>
</Lems>


Found following parameters:
['I', 'a', 'alpha', 'b', 'beta', 'c', 'd', 'e', 'f', 'g', 'gamma', 'tau']
<?xml version="1.0" ?>
<Lems xmlns="http://www.neuroml.org/lems/0.7.6" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schemaLocation="http://www.neuroml.org/lems/0.7.6 https://raw.githubusercontent.com/LEMS/LEMS/development/Schemas/LEMS/LEMS_v0.7.6.xsd">
    <Component I="0.0" a="-2.0" alpha="1.0" b="-10.0" beta="1.0" c="0.0" d="0.02" e="3.0" f="1.0" g="0.0" gamma="1.0" tau="1.0"/>
</Lems>


Found following parameters:
[]
<?xml version="1.0" ?>
<Lems xmlns="http://www.neuroml.org/lems/0.7.6" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:schem

In [50]:
f = h5py.File('C:\\Users\\dinar\\Desktop\\gsoc\\data\\simulations\\1\\datatypes\\Connectivity_6b28b6c61836452bb52cbd18538a7775.h5')
f.keys()

<KeysViewHDF5 ['areas', 'centres', 'cortical', 'hemispheres', 'orientations', 'region_labels', 'tract_lengths', 'weights']>

In [14]:
np.array([str(x).strip("b'") for x in f])

array(['rA1', 'rA2', 'rAMYG', 'rCCA', 'rCCP', 'rCCR', 'rCCS', 'rFEF',
       'rG', 'rHC', 'rIA', 'rIP', 'rM1', 'rPCI', 'rPCIP', 'rPCM', 'rPCS',
       'rPFCCL', 'rPFCDL', 'rPFCDM', 'rPFCM', 'rPFCORB', 'rPFCPOL',
       'rPFCVL', 'rPHC', 'rPMCDL', 'rPMCM', 'rPMCVL', 'rS1', 'rS2',
       'rTCC', 'rTCI', 'rTCPOL', 'rTCS', 'rTCV', 'rV1', 'rV2', 'rCC',
       'lA1', 'lA2', 'lAMYG', 'lCCA', 'lCCP', 'lCCR', 'lCCS', 'lFEF',
       'lG', 'lHC', 'lIA', 'lIP', 'lM1', 'lPCI', 'lPCIP', 'lPCM', 'lPCS',
       'lPFCCL', 'lPFCDL', 'lPFCDM', 'lPFCM', 'lPFCORB', 'lPFCPOL',
       'lPFCVL', 'lPHC', 'lPMCDL', 'lPMCM', 'lPMCVL', 'lS1', 'lS2',
       'lTCC', 'lTCI', 'lTCPOL', 'lTCS', 'lTCV', 'lV1', 'lV2', 'lCC'],
      dtype='<U7')

In [6]:
import h5py


data = h5py.File('C:\\Users\\dinar\\Desktop\\gsoc\\data\\simulations\\1\\Linear_f639e3b95c48436dae5ffaa29072963e.h5')
dir(data)

['_MutableMapping__marker',
 '__abstractmethods__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__nonzero__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_d',
 '_e',
 '_gcpl_crt_order',
 '_id',
 '_ipython_key_completions_',
 '_lapl',
 '_lcpl',
 '_libver',
 'attrs',
 'build_virtual_dataset',
 'clear',
 'close',
 'copy',
 'create_dataset',
 'create_dataset_like',
 'create_group',
 'create_virtual_dataset',
 'driver',
 'file',
 'filename',
 'flush',
 'get',
 'id',
 'items',
 'key

In [6]:
data.keys()

<KeysViewHDF5 ['a', 'b']>

In [77]:
data['a'][:]

array([0.05])

In [14]:
data = h5py.File('C:\\Users\\dinar\\Desktop\\gsoc\\data\\simulations\\1\\datatypes\\Connectivity_6b28b6c61836452bb52cbd18538a7775.h5')
data.keys()

<KeysViewHDF5 ['areas', 'centres', 'cortical', 'hemispheres', 'orientations', 'region_labels', 'tract_lengths', 'weights']>

In [82]:
list(set(['centres', 'weights', 'tract_lengths']).intersection(set(data.keys())))

['weights', 'tract_lengths', 'centres']

In [95]:
data['region_labels'][:]

array([b'rA1', b'rA2', b'rAMYG', b'rCCA', b'rCCP', b'rCCR', b'rCCS',
       b'rFEF', b'rG', b'rHC', b'rIA', b'rIP', b'rM1', b'rPCI', b'rPCIP',
       b'rPCM', b'rPCS', b'rPFCCL', b'rPFCDL', b'rPFCDM', b'rPFCM',
       b'rPFCORB', b'rPFCPOL', b'rPFCVL', b'rPHC', b'rPMCDL', b'rPMCM',
       b'rPMCVL', b'rS1', b'rS2', b'rTCC', b'rTCI', b'rTCPOL', b'rTCS',
       b'rTCV', b'rV1', b'rV2', b'rCC', b'lA1', b'lA2', b'lAMYG', b'lCCA',
       b'lCCP', b'lCCR', b'lCCS', b'lFEF', b'lG', b'lHC', b'lIA', b'lIP',
       b'lM1', b'lPCI', b'lPCIP', b'lPCM', b'lPCS', b'lPFCCL', b'lPFCDL',
       b'lPFCDM', b'lPFCM', b'lPFCORB', b'lPFCPOL', b'lPFCVL', b'lPHC',
       b'lPMCDL', b'lPMCM', b'lPMCVL', b'lS1', b'lS2', b'lTCC', b'lTCI',
       b'lTCPOL', b'lTCS', b'lTCV', b'lV1', b'lV2', b'lCC'], dtype=object)

Creating folder `../output\sub-P0kvO`
Creating folder `../output\sub-P0kvO\net`
Creating folder `../output\sub-P0kvO\spatial`
Creating folder `../output\sub-P0kvO\ts`


In [97]:
data['weights'][:]

array([[2., 2., 0., ..., 0., 0., 0.],
       [3., 2., 0., ..., 0., 0., 0.],
       [0., 0., 2., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 2., 2., 0.],
       [0., 0., 0., ..., 3., 2., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [15]:
data['centres'][:]

array([[ -9.885591, -47.084818,  -3.13936 ],
       [ -2.605247, -55.324507,  -7.065423],
       [  6.489431, -11.670519, -30.851544],
       [ 48.734149,  -2.004701,   8.019123],
       [-12.1714  ,  -6.494719,  25.361539],
       [-38.1525  , -13.054752,  -5.607666],
       [ 30.150795,  -4.286333, -31.136279],
       [ 30.400644, -33.753689,  34.832572],
       [ 36.159029, -35.084312, -18.938829],
       [-12.502286, -24.875165, -20.872351],
       [ 23.242996, -42.031295, -22.110624],
       [  9.822772, -44.114976, -18.760442],
       [  8.003023, -32.833685,  38.712349],
       [-22.152851, -56.734891,  16.852029],
       [-51.151034, -35.899675,  25.679908],
       [-43.927847,  -8.226723,  27.584594],
       [-30.092058, -33.534852,  43.097159],
       [ 54.048553, -37.427609,  12.535183],
       [ 59.995574, -14.366518,  26.025739],
       [ 59.612735,  -2.407101,  -0.844581],
       [ 59.286775,  -2.547348, -21.924281],
       [ 47.987255, -16.887697, -39.699154],
       [ 7

In [60]:
loadmat("C:\\Users\\dinar\\Desktop\\simulated_data\\1\\MSE\\P2_log_SC_long_bold_csf_0.1_K11_0.1_K12_0.1.mat")

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Thu Apr 30 16:48:01 2020',
 '__version__': '1.0',
 '__globals__': [],
 'subs_data': array([[[[  -3.84944715,   -1.96095335,   -2.69455228],
          [  -4.04889455,   -1.98311898,   -2.33277659],
          [  -3.94176721,   -2.05415186,   -2.87601393],
          ...,
          [  -4.29774122,   -1.85233491,   -2.90531064],
          [  -4.49811713,   -1.65955181,   -2.68645867],
          [  -3.88675838,   -2.01378071,   -2.872956  ]],
 
         [[  -3.69193033,   -1.68575999,   -2.23651396],
          [  -3.94768865,   -1.94424875,   -2.54331918],
          [  -3.82984156,   -2.33304909,   -2.40755582],
          ...,
          [  -4.06293296,   -1.47831599,   -2.22740778],
          [  -3.79118515,   -1.82996363,   -2.39362346],
          [  -4.25364555,   -1.90993204,   -2.78710827]]],
 
 
        [[[ -65.38614263,  -30.49315838,  -45.18039752],
          [ -64.89151942,  -30.94269337,  -42.84714715],
          [ -6

In [16]:
import mat73
from scipy.io import loadmat

In [10]:
data = loadmat('C:\\Users\\dinar\\Desktop\\simulated_data\\1\\DBS\\newest_timeseries\\GPi_stim\\tvb_timeseries.mat',
               squeeze_me=True)
data['dims']

array(['Time          ', 'State Variable', 'Region        ',
       'Neurons       '], dtype='<U14')

In [53]:
def find_mat_array(mat):
    data = []

    for k, v in mat.items():
        if type(v) not in [bytes, str, list]:
            data.append(k)

    return data


find_mat_array(data)

['dims', 'attrs', 'data', 'coords', 'transient']

In [55]:
data

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Thu Mar 25 16:23:45 2021',
 '__version__': '1.0',
 '__globals__': [],
 'dims': array(['Time          ', 'State Variable', 'Region        ',
        'Neurons       '], dtype='<U14'),
 'attrs': None,
 'data': array([[[9.24158645e-02, 8.96056728e-02, 4.89930394e-02, ...,
          1.01463449e-01, 1.55864629e-02, 5.87542230e-02],
         [1.71684241e+00, 1.73930070e+00, 1.80429717e+00, ...,
          0.00000000e+00, 2.69357687e-04, 1.10159536e-02],
         [1.31146622e+00, 1.31892330e+00, 1.34228813e+00, ...,
          8.75345250e-01, 4.67516590e-01, 6.83007184e-01],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [3.39237277e-01, 3.34011871e-01, 5.75926660e-01, ...,
          3.22753203e-01, 3.02678035e-01, 3.14682057e-01]],
 
        [[9.15398269e-02, 9.09408421e-02, 4.88707572e-02, ...,
          1.00317053e-01, 1.22318579e-02, 5.47590293

['ALS_binary', 'ALS_weighted', 'HC_binary', 'HC_weighted', 'coord', 'regionDescriptions']


In [54]:
type(data['coords'])

numpy.ndarray

In [11]:
data['attrs']

In [12]:
data['data']

array([[[9.24158645e-02, 8.96056728e-02, 4.89930394e-02, ...,
         1.01463449e-01, 1.55864629e-02, 5.87542230e-02],
        [1.71684241e+00, 1.73930070e+00, 1.80429717e+00, ...,
         0.00000000e+00, 2.69357687e-04, 1.10159536e-02],
        [1.31146622e+00, 1.31892330e+00, 1.34228813e+00, ...,
         8.75345250e-01, 4.67516590e-01, 6.83007184e-01],
        [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
         0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
        [3.39237277e-01, 3.34011871e-01, 5.75926660e-01, ...,
         3.22753203e-01, 3.02678035e-01, 3.14682057e-01]],

       [[9.15398269e-02, 9.09408421e-02, 4.88707572e-02, ...,
         1.00317053e-01, 1.22318579e-02, 5.47590293e-02],
        [1.71147984e+00, 1.73721764e+00, 1.80401869e+00, ...,
         4.86993512e-03, 5.56826672e-04, 6.19613245e-03],
        [1.31028333e+00, 1.31882550e+00, 1.34324129e+00, ...,
         9.04258234e-01, 4.82442158e-01, 6.64427306e-01],
        [0.00000000e+00, 0.00000000e+0

In [18]:
dir(data['coords'])

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',

In [36]:
dir(data['coords'])

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',

In [37]:
data['transient']

500.0

In [38]:
data

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Thu Mar 25 16:23:45 2021',
 '__version__': '1.0',
 '__globals__': [],
 'dims': array(['Time          ', 'State Variable', 'Region        ',
        'Neurons       '], dtype='<U14'),
 'attrs': None,
 'data': array([[[9.24158645e-02, 8.96056728e-02, 4.89930394e-02, ...,
          1.01463449e-01, 1.55864629e-02, 5.87542230e-02],
         [1.71684241e+00, 1.73930070e+00, 1.80429717e+00, ...,
          0.00000000e+00, 2.69357687e-04, 1.10159536e-02],
         [1.31146622e+00, 1.31892330e+00, 1.34228813e+00, ...,
          8.75345250e-01, 4.67516590e-01, 6.83007184e-01],
         [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
          0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
         [3.39237277e-01, 3.34011871e-01, 5.75926660e-01, ...,
          3.22753203e-01, 3.02678035e-01, 3.14682057e-01]],
 
        [[9.15398269e-02, 9.09408421e-02, 4.88707572e-02, ...,
          1.00317053e-01, 1.22318579e-02, 5.47590293

In [46]:
dir(data['coords'])

['T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_finalize__',
 '__array_function__',
 '__array_interface__',
 '__array_prepare__',
 '__array_priority__',
 '__array_struct__',
 '__array_ufunc__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__class_getitem__',
 '__complex__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__divmod__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__ilshift__',
 '__imatmul__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',

In [52]:
data['coords'].base[0][0]

array(('State Variable', None, array(['S   ', 'Rint', 'R   ', 'Rin ', 'I   '], dtype='<U4')),
      dtype=[('dims', 'O'), ('attrs', 'O'), ('data', 'O')])

In [20]:
mat73.loadmat("C:\\Users\\dinar\\Desktop\\simulated_data\\1\\berlin-unimanual\\AA_20120815_analysis_.mat").keys()

dict_keys(['MSE', 'PSP_baseline', 'dominant_rhythm'])